In [ ]:
from glob import glob
from pathlib import Path
from face_alignment import align
from inference import load_pretrained_model, to_input

# architecture = 'ir_50'
# pretrained_path = '/home/s3-subtitle-download/pretrained_models/AdaFace/adaface_ir50_ms1mv2.ckpt'
# model = load_pretrained_model(architecture, pretrained_path)

# ONNX

### ONNX batch


In [ ]:
from PIL import Image
import torch
import onnxruntime as ort

# ONNX_PATH = 'adaface.onnx'
ONNX_PATH = '/home/s3-subtitle-download/pretrained_models/AdaFace/adaface.onnx'

SIZE = (112, 112)

is_face = False
paths = sorted(glob('_test/*.jpg'))

# is_face = True  # もともとfaceがクロップされた画像か（クロップされた画像の場合, 顔検出がうまくいかないことがある）
# paths = sorted(glob('_test/cropped_face/*.jpg'), key=lambda x: int(Path(x).stem))

faces = []
for path in paths:
    if is_face:
        face_pil_img = Image.open(path).convert('RGB').resize(SIZE)
    else:
        face_pil_img = align.get_aligned_face(path)
    bgr_input = to_input(face_pil_img)
    faces.append(bgr_input)

faces = torch.cat(faces, dim=0)

ort_session = ort.InferenceSession(ONNX_PATH)
input_name = ort_session.get_inputs()[0].name
input_data = faces.numpy()
features = ort_session.run(None, {input_name: input_data})
features = features[0]

names = [Path(path).stem for path in paths]
features = {name: torch.tensor(feature).unsqueeze(0) for name, feature in zip(names, features)}

# Checkpoint

### single image


In [ ]:
# features = {}
# for path in glob('_test/*.jpg'):
#     aligned_rgb_img = align.get_aligned_face(path)
#     bgr_input = to_input(aligned_rgb_img)
#     feature, _ = model(bgr_input)
#     features[Path(path).stem] = feature

### multiple images, bath processing


In [ ]:
# # multiple images, bath processing
# import torch
# paths = glob('_test/*.jpg')

# faces = []
# for path in paths:
#     aligned_rgb_img = align.get_aligned_face(path)
#     bgr_input = to_input(aligned_rgb_img)
#     faces.append(bgr_input)

# faces = torch.cat(faces, dim=0)
# features = model(faces)[0]

# names = [Path(path).stem for path in paths]
# features = {name: feature.unsqueeze(0) for name, feature in zip(names, features)}

# Calculate Similarity


In [ ]:
# calculate similarity
import torch.nn.functional as F
for name1, feature1 in features.items():
    for name2, feature2 in features.items():
        print(f'{name1:8}, {name2:8}\t{F.cosine_similarity(feature1, feature2).item():.4f}')

# Comparison to FaceRecognition


In [ ]:
# import torch
# import face_recognition
# fr_features = {}
# for path in glob('_test/*.jpg'):
#     img = face_recognition.load_image_file(path)
#     ft = face_recognition.face_encodings(img)
#     fr_features[Path(path).stem] = torch.tensor(ft)
# # 
# for name1, feature1 in fr_features.items():
#     for name2, feature2 in fr_features.items():
#         print(f'{name1:8}, {name2:8}\t{F.cosine_similarity(feature1, feature2).item():.4f}')